In [2]:
import mysql.connector
import pandas as pd
config = {
    "host": "localhost",
    "user": "jongwoonglee",
    "password": "0075",
    "database": "online_retail_dataset",
    "port": 3307
}

try:
    # MySQL 연결 생성
    connection = mysql.connector.connect(**config)
    cursor = connection.cursor()

    # 쿼리 실행
#     create_view_query = """
#     CREATE VIEW extracted_sales_raw_view AS (
#         SELECT
#             YEAR(InvoiceDate) AS year,
#             MONTH(InvoiceDate) AS month,
#             customerid,
#             country,
#             SUM(quantity) AS quantity
#         FROM `sales_raw`
#         WHERE customerid IS NOT NULL 
#         GROUP BY 1, 2, 3, 4
#         HAVING SUM(quantity) > 0
#     )
#     """
    select_view_query = "SELECT * FROM extracted_sales_raw_view LIMIT 999999;"

#     # CREATE VIEW 쿼리 실행
#     cursor.execute(create_view_query)
#     connection.commit()

    # SELECT 쿼리 실행
    cursor.execute(select_view_query)
    result = cursor.fetchall()

    # 결과를 데이터프레임으로 변환
    columns = [column[0] for column in cursor.description]
    raw_data = pd.DataFrame(result, columns=columns)
    
    print(raw_data) 

        

except mysql.connector.Error as err:
    print("Error:", err)

finally:
    # 연결 종료
    if 'connection' in locals() and connection.is_connected():
        cursor.close()
        connection.close()

       year  month  customerid         country quantity
0      2010     12     17850.0  United Kingdom     1733
1      2010     12     13047.0  United Kingdom      101
2      2010     12     12583.0          France      449
3      2010     12     13748.0  United Kingdom       80
4      2010     12     15100.0  United Kingdom       45
...     ...    ...         ...             ...      ...
12979  2011     12     12713.0         Germany      508
12980  2011     12     17581.0  United Kingdom      516
12981  2011     12     15804.0  United Kingdom      278
12982  2011     12     13113.0  United Kingdom       66
12983  2011     12     12680.0          France      105

[12984 rows x 5 columns]


In [3]:
raw_data

,year,month,customerid,country,quantity
0,2010,12,17850.0,United Kingdom,1733
1,2010,12,13047.0,United Kingdom,101
2,2010,12,12583.0,France,449
3,2010,12,13748.0,United Kingdom,80
4,2010,12,15100.0,United Kingdom,45
...,...,...,...,...,...
12979,2011,12,12713.0,Germany,508
12980,2011,12,17581.0,United Kingdom,516
12981,2011,12,15804.0,United Kingdom,278
12982,2011,12,13113.0,United Kingdom,66


In [15]:
raw_data['customerid'].value_counts()

15358.0    13
16033.0    13
13694.0    13
15769.0    13
17576.0    13
           ..
14280.0     1
14816.0     1
12974.0     1
16448.0     1
12713.0     1
Name: customerid, Length: 4327, dtype: int64

In [14]:
result_counts = raw_data['customerid'].value_counts()

# 10 이상인 값들 추출하여 리스트에 넣기
values_over_10 = [value for value, count in result_counts.items() if count >= 10]
print(12984 /len(values_over_10))
# print(values_over_10)

71.73480662983425


In [10]:
raw_data[raw_data['customerid']==15358.0]

,year,month,customerid,country,quantity
522,2010,12,15358.0,United Kingdom,475
958,2011,1,15358.0,United Kingdom,1025
2011,2011,2,15358.0,United Kingdom,374
2553,2011,3,15358.0,United Kingdom,827
3764,2011,4,15358.0,United Kingdom,503
4785,2011,5,15358.0,United Kingdom,455
5770,2011,6,15358.0,United Kingdom,307
7032,2011,7,15358.0,United Kingdom,141
7568,2011,8,15358.0,United Kingdom,707
8578,2011,9,15358.0,United Kingdom,933


In [5]:
# xlsx로 내보내기
output_path = 'C:/Users/dlwhd/Downloads/TIL/SQL을 이용한 Retention 분석/Preprocessed_In_SQL_data.xlsx'
raw_data.to_excel(output_path, index=False)  # index=False로 설정하면 인덱스를 저장하지 않음

## **여기부터는 파이썬에서 전처리 하는 것**

#### 음수의 Quantity (환불) 제거

In [ ]:
raw_data[raw_data['Quantity'] < 0 ]

In [ ]:
idx = raw_data[raw_data['Quantity'] < 0].index

Delete_Negative_data = raw_data.drop(idx,inplace=False)
Delete_Negative_data

In [ ]:
Delete_Negative_data.info()

In [ ]:
Delete_Negative_data.isnull().sum()

#### 결측치 없음

In [ ]:
Delete_Negative_data

In [ ]:
Delete_Negative_data['CustomerID'] = Delete_Negative_data['CustomerID'].astype('int')
Delete_Negative_data['Quantity'] = Delete_Negative_data['Quantity'].astype('int')
Delete_Negative_data

In [ ]:
query = """
    SELECT DISTINCT CustomerID, COUNT(DISTINCT(InvoiceDate)) AS OrderCount
    FROM `sales_raw`
    GROUP BY CustomerID
    ORDER BY OrderCount DESC
    LIMIT 999999;
"""

data = pd.read_sql(query, con=db_connection)
data['CustomerID'] = data['CustomerID'].astype('int')
data['OrderCount'] = data['OrderCount'].astype('int')
data

In [ ]:
Hyper_CustomerID = []
Super_CustomerID = []
General_CustomerID = []

for i in range(len(data)-1):
    order_count = data.loc[i, 'OrderCount']  # 현재 행의 OrderCount 값을 가져옴
    if order_count > 39:
        Hyper_CustomerID.append(data.loc[i, 'CustomerID'])
    elif order_count > 14:
        Super_CustomerID.append(data.loc[i, 'CustomerID'])
    elif order_count > 0:
        General_CustomerID.append(data.loc[i, 'CustomerID'])

print("Hyper_CustomerID 리스트 : ", Hyper_CustomerID)
print("Super_CustomerID 리스트 : ", Super_CustomerID)
print("General_CustomerID 리스트 : ", General_CustomerID)

In [ ]:
Hyper_Customer_Data = Delete_Negative_data[Delete_Negative_data['CustomerID'].isin(Hyper_CustomerID)]
Super_CustomerData = Delete_Negative_data[Delete_Negative_data['CustomerID'].isin(Super_CustomerID)]
General_CustomerData = Delete_Negative_data[Delete_Negative_data['CustomerID'].isin(General_CustomerID)]

In [ ]:
# YYYY-MM-DD 형태로 통일하고 Datatime 자료형으로 바꾸기 
import pandas as pd
from datetime import datetime

# 문자열을 datetime 객체로 변환하는 함수
def convert_to_datetime(date_str):
    return datetime.strptime(date_str, "%Y-%m-%d %H:%M").strftime("%Y-%m-%d")

# apply 함수를 사용하여 각 행에 대해 변환 적용
Hyper_Customer_Data['InvoiceDate'] = Hyper_Customer_Data['InvoiceDate'].apply(convert_to_datetime)
Super_CustomerData['InvoiceDate'] = Super_CustomerData['InvoiceDate'].apply(convert_to_datetime)
General_CustomerData['InvoiceDate'] = General_CustomerData['InvoiceDate'].apply(convert_to_datetime)
Hyper_Customer_Data['InvoiceDate']

이제 첫 구매년월을 기준으로 코호트를 나눈다. 그 고객들이 N달 후에도 이용하는지 알아보고 리텐션 출력값을 얻는다.

In [ ]:
# xlsx로 내보내기
output_path = 'C:/Users/dlwhd/Downloads/TIL/SQL을 이용한 Retention 분석/Hyper_Customer_Data.xlsx'
Hyper_Customer_Data.to_excel(output_path, index=False)  # index=False로 설정하면 인덱스를 저장하지 않음
output_path = 'C:/Users/dlwhd/Downloads/TIL/SQL을 이용한 Retention 분석/Super_CustomerData.xlsx'
Super_CustomerData.to_excel(output_path, index=False)
output_path = 'C:/Users/dlwhd/Downloads/TIL/SQL을 이용한 Retention 분석/General_CustomerData.xlsx'
General_CustomerData.to_excel(output_path, index=False)

In [ ]:
# xlsx로 내보내기
output_path = 'C:/Users/dlwhd/Downloads/TIL/SQL을 이용한 Retention 분석/Preprocessed_OrderCount_data.xlsx'
data.to_excel(output_path, index=False)  # index=False로 설정하면 인덱스를 저장하지 않음

In [ ]:
# xlsx로 내보내기
output_path = 'C:/Users/dlwhd/Downloads/TIL/SQL을 이용한 Retention 분석/Preprocessed_data.xlsx'
Delete_Negative_data.to_excel(output_path, index=False)  # index=False로 설정하면 인덱스를 저장하지 않음

**아래에서 나오는 중복제거 로직은, 엄밀히는 중복값이 아니지만, 고객별 그 날 주문 건수로 파악하기 위해서 제거하는 중복값 제거이다.**


In [ ]:
# 중복된 값이 있는 컬럼들로 그룹화
groups = Delete_Negative_data.groupby(['InvoiceNo', 'Description', 'InvoiceDate', 'CustomerID'])

# 그룹화된 데이터에서 StockCode 값이 다른 행만 추출
Day_Unique_Purchase_Record = groups.filter(lambda group: group['StockCode'].nunique() > 1)
Day_Unique_Purchase_Record